In [ ]:
import pandas as pd
import altair as alt

# Load data
url = 'https://raw.githubusercontent.com/YOUR_USERNAME/your-repo/main/building_inventory.csv'
df = pd.read_csv(url)

# Data transformation
map_data = df.dropna(subset=['Square Footage', 'City', 'Bldg Status']).groupby(['City', 'Bldg Status']).agg(
    Total_Square_Footage=('Square Footage', 'sum')
).reset_index()

# Interactive map
map_chart = alt.Chart(map_data).mark_circle().encode(
    longitude=alt.Longitude('Longitude:Q'),
    latitude=alt.Latitude('Latitude:Q'),
    size=alt.Size('Total_Square_Footage:Q', title='Square Footage'),
    color=alt.Color('Bldg Status:N', title='Building Status'),
    tooltip=['City', 'Total_Square_Footage', 'Bldg Status']
).properties(
    width=800,
    height=500,
    title="Building Distribution in Illinois"
).interactive()

map_chart.show()
# Data transformation
bar_data = df.dropna(subset=['Square Footage', 'Year Acquired']).groupby(['Bldg Status', 'Year Acquired']).agg(
    Total_Square_Footage=('Square Footage', 'sum')
).reset_index()

# Interactive elements
status_dropdown = alt.binding_select(options=bar_data['Bldg Status'].unique(), name="Building Status: "
status_selection = alt.selection_single(fields=['Bldg Status'], bind=status_dropdown)

year_slider = alt.binding_range(min=bar_data['Year Acquired'].min(), max=bar_data['Year Acquired'].max(), name="Year: "
year_selection = alt.selection_single(fields=['Year Acquired'], bind=year_slider)

# Bar chart
bar_chart = alt.Chart(bar_data).mark_bar().encode(
    x=alt.X('Year Acquired:Q', title='Year Acquired'),
    y=alt.Y('Total_Square_Footage:Q', title='Total Square Footage'),
    color=alt.Color('Bldg Status:N', title='Building Status'),
    tooltip=['Year Acquired', 'Total_Square_Footage', 'Bldg Status']
).add_selection(
    status_selection,
    year_selection
).transform_filter(
    status_selection
).transform_filter(
    year_selection
).properties(
    width=800,
    height=400,
    title="Square Footage by Building Status Over Time"
)

bar_chart.show()
